In [5]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import datetime
from jugaad_data.nse import stock_df
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

In [6]:
# Load environment variables from .env file
load_dotenv()
# Access the database URL
db_url = os.getenv('DB_URL')

In [22]:
# Create connect sql database
def create_connection_database(db_url):
    # Database connection string
    db_url = db_url
    # Create an SQLAlchemy engine
    return create_engine(db_url)

# read data from the PostgreSQL table
try:
   with create_connection_database(db_url).connect() as connection:
    table_name = "nifty_top_500_stocks"
    query = f"SELECT * from {table_name};"
    df = pd.read_sql_query(query, con=create_connection_database(db_url))
    print("data pulled from sql table")
except Exception as e:
    print(f"Error: {e}")


def pull_stock_data(stock_symbol:str,start_date:datetime) -> pd.DataFrame:
    today = datetime.date.today()
    print(today)
    data = stock_df(symbol=stock_symbol, from_date=start_date,
            to_date=today, series="EQ")
    
    return data

# check last updated date for each stock and add data till current date
master_data = pd.DataFrame()
for symbol in df.SYMBOL.unique():
    start_date = df[df.SYMBOL == symbol].DATE.max().date()
    try:
       
        data = pull_stock_data(stock_symbol=symbol,start_date=start_date+datetime.timedelta(days=1))
        
        master_data = pd.concat([master_data,data],axis=0)
    except:
        pass

print("latest stocks data pulled")

# add new data in database
try:
   with create_connection_database(db_url).connect() as connection:
    table_name = "nifty_top_500_stocks"
    master_data.to_sql(table_name,create_connection_database(db_url),index=False,if_exists='append')
    print(f"{len(master_data)} rows successfully added in {table_name}")
except Exception as e:
    print(f"Error: {e}")

data pulled from sql table
360ONE 2024-12-20
2024-12-24


,DATE,SERIES,OPEN,HIGH,LOW,PREV. CLOSE,LTP,CLOSE,VWAP,52W H,52W L,VOLUME,VALUE,NO OF TRADES,SYMBOL
0,2024-12-23,EQ,1238.05,1280.75,1208.95,1249.5,1225.1,1232.7,1229.79,1280.75,591.5,499836,614693051.9,40886,360ONE


,DATE,SERIES,OPEN,HIGH,LOW,PREV. CLOSE,LTP,CLOSE,VWAP,52W H,52W L,VOLUME,VALUE,NO OF TRADES,SYMBOL
0,2024-12-23,EQ,1238.05,1280.75,1208.95,1249.5,1225.1,1232.7,1229.79,1280.75,591.5,499836,614693051.9,40886,360ONE


3MINDIA 2024-12-20
2024-12-24


,DATE,SERIES,OPEN,HIGH,LOW,PREV. CLOSE,LTP,CLOSE,VWAP,52W H,52W L,VOLUME,VALUE,NO OF TRADES,SYMBOL
0,2024-12-23,EQ,31100.0,31146.6,30522.0,30813.85,30739.85,30676.35,30746.8,40856.5,28498.95,2004,61616584.8,1202,3MINDIA


...

,DATE,SERIES,OPEN,HIGH,LOW,PREV. CLOSE,LTP,CLOSE,VWAP,52W H,52W L,VOLUME,VALUE,NO OF TRADES,SYMBOL
0,2024-12-23,EQ,978.75,982.0,962.1,973.5,971.8,973.35,973.94,1324.3,666.75,758127,7.383703e+08,36672,ZYDUSLIFE


ECLERX 2024-12-20
2024-12-24


,DATE,SERIES,OPEN,HIGH,LOW,PREV. CLOSE,LTP,CLOSE,VWAP,52W H,52W L,VOLUME,VALUE,NO OF TRADES,SYMBOL
0,2024-12-23,EQ,3637.9,3692.75,3596.2,3608.95,3610.0,3611.9,3643.0,3877.0,2114.0,107112,390209113.5,17750,ECLERX


,DATE,SERIES,OPEN,HIGH,LOW,PREV. CLOSE,LTP,CLOSE,VWAP,52W H,52W L,VOLUME,VALUE,NO OF TRADES,SYMBOL
0,2024-12-23,EQ,3637.9,3692.75,3596.2,3608.95,3610.0,3611.9,3643.0,3877.0,2114.0,107112,390209113.5,17750,ECLERX


latest stocks data pulled
1101 rows successfully added in nifty_top_500_stocks


In [10]:
datetime.date.today()

datetime.date(2024, 12, 24)

In [20]:
t =df[df.SYMBOL == "360ONE"].DATE.max().date()
t

datetime.date(2024, 12, 20)

In [21]:
stock_df(symbol="360ONE", from_date=t,
            to_date=datetime.date(2024, 12, 24), series="EQ")

,DATE,SERIES,OPEN,HIGH,LOW,PREV. CLOSE,LTP,CLOSE,VWAP,52W H,52W L,VOLUME,VALUE,NO OF TRADES,SYMBOL
0,2024-12-23,EQ,1238.05,1280.75,1208.95,1249.50,1225.10,1232.7,1229.79,1280.75,591.5,499836,6.146931e+08,40886,360ONE
1,2024-12-20,EQ,1225.35,1260.00,1210.35,1225.35,1223.15,1249.5,1250.26,1270.00,591.5,1452337,1.815797e+09,53665,360ONE
